In [1]:
#import libraries
import os
import pyodbc
import numpy as np
import pandas as pd
import glob

In [32]:
#find csv files in my current working directory

csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [ ]:
csv_files

In [ ]:
df = {}
for file in csv_files:
    df[file] = pd.read_csv(file)
    print(file)

In [35]:
def change_column_name(data, column_name, target_name):
    data.columns = data.columns.str.replace(column_name, target_name)
    return data

In [36]:
def do_cleaning_nan(data, value_for_replacing = 0):
    cleaned_data = data.fillna(value_for_replacing)
    return cleaned_data

In [ ]:
for k in csv_files:
    dataframe = df[k]
    df[k] = do_cleaning_nan(df[k],0)
    print(df[k])

In [ ]:
for k in csv_files:
    dataframe = df[k]
    cleaned_data = change_column_name(df[k], 'Unnamed: 0', 't_id')
    cleaned_data = change_column_name(df[k], 'Time', 'p_time')
    print(df[k])

In [ ]:
for k in csv_files:
    dataframe = df[k]
    dataframe.columns = [x.replace(" ","_").replace("?","") \
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                     .replace(")","").replace(r"(","").replace("$","") for x in dataframe.columns]

    print(dataframe.columns)
    
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedate64[ns]': 'varchar'
}

col_str =", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements))) 
print(col_str)

In [ ]:
pyodbc.drivers()

conn = pyodbc.connect(
    Trusted_connected="Yes",
    Driver='{SQL Server}',
    Server="LAPTOP-DJB44MI5\MSSQLSERVER02",
    Database="Test"
)

conn.setdecoding(pyodbc.SQL_CHAR, encoding='latin1')
conn.setencoding('latin1')

cursor = conn.cursor()

print('open database successfully')

In [ ]:
cursor.execute("drop table if exists traffic_bonn;")

In [42]:
cursor.execute('''
		CREATE TABLE traffic_bonn (
			t_id int,
			opath_id int,
			source varchar(max),
			time varchar(max),
			speed float,
            traffic float,
            incidentType varchar(max),
            incidentID float,
            incidentComments varchar(max),
            Construction_Blockage varchar(max),
            Consumptionl_h float,
            Rpmu_min float,
            Throttle_Position float,
            CO2kg_h float,
            vehicle_id varchar(max),
            id int,
            opath varchar(max),
            p_time varchar(max),
            weather_condition varchar(max)
            
            
            
	    	)
              ''')

conn.commit()

In [ ]:
for k in csv_files:
    dataframe = df[k]
    
    for row in dataframe.itertuples():
            
        cursor.execute('''
        INSERT INTO Test.dbo.traffic_bonn(t_id, opath_id, source, time, speed, traffic, incidentType, incidentID, incidentComments, Construction_Blockage, Consumptionl_h, Rpmu_min, Throttle_Position, CO2kg_h, vehicle_id, id, opath, p_time, weather_condition)
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
                   row.t_id,
                   row.opath_id,
                   row.source,
                   row.time,
                   row.speed,
                   row.traffic,
                   row.incidentType,
                   row.incidentID,
                   row.incidentComments,
                   row.Construction_Blockage,
                   row.Consumptionl_h,
                   row.Rpmu_min,
                   row.Throttle_Position,
                   row.CO2kg_h,
                   row.vehicle_id,
                   row.id,
                   row.opath,
                   row.p_time,
                   row.weather_condition
   )
    
    conn.commit()
    print('Data Inserted')
